# Ollama Example in langchain
This is a LLM example with a ollama local model.
It's free and fast (I can easily run it on my GTX1660 Super)
Code  
https://python.langchain.com/docs/get_started/quickstart

Ollama is a model manager, which allows you to download access models via Ollama

Ollama Server  
https://ollama.ai/download/linux  
Ollama Model  
https://ollama.ai/library  

In [1]:
model_id="llama2-uncensored"
# !ollama pull $model_id

# NOTE: the model stores in ~/.ollama/models

In [2]:
from langchain_community.llms import Ollama
llm = Ollama(model=model_id)

## Prompt (with model)

In [3]:
%%time
# llm.invoke("how can langsmith help with testing?") <-- without streaming

# streaming
for chunk in llm.stream("how can langsmith help with testing?"):
    print(chunk, end="", flush=True)
print("\n")

Langsmith is a natural language processing (NLP) toolkit that provides various features for text analysis and language understanding. It offers several testing functions, including sentiment analysis, topic modeling, and named entity recognition, which can be useful in various applications such as customer feedback analysis, news article classification, and product recommendation systems. Additionally, Langsmith provides pre-trained models for various languages, which can be trained on large datasets to improve the accuracy of the NLP tasks performed by the toolkit.

CPU times: user 161 ms, sys: 65.4 ms, total: 226 ms
Wall time: 14.7 s


## Prompt (with LCEL)

Note that we pass an "dict" into invoke/stream/astream, not str

In [4]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])

In [5]:
from langchain_core.output_parsers import StrOutputParser
chain = prompt | llm | StrOutputParser()
print(chain)

first=ChatPromptTemplate(input_variables=['input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are world class technical documentation writer.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))]) middle=[Ollama(model='llama2-uncensored')] last=StrOutputParser()


In [6]:
%%time
# no streaming
# chain.invoke({"input": "how can langsmith help with testing?"})

## async streaming
# async for chunk in chain.astream({"input": "how can langsmith help with testing?"}):
#     print(chunk, end="", flush=True)

# stream
for chunk in chain.stream({"input": "how can langsmith help with testing?"}):
    print(chunk, end="", flush=True)
print("\n")

Langsmith provides a wide range of AI-powered tools to assist in the process of writing technical documents, such as automated document summarization, grammar checking, and error correction. These features can be extremely helpful for testing your technical documentation to ensure that it is clear, accurate, and free from errors before publishing or presenting it to clients.

CPU times: user 92 ms, sys: 19.8 ms, total: 112 ms
Wall time: 9.59 s


## RAG

In [7]:
from langchain.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://blog.google/outreach-initiatives/sustainability/report-ai-sustainability-google-cop28/")
docs = loader.load()
print(docs[0].__dict__.keys())
print(docs[0].metadata)

dict_keys(['page_content', 'metadata', 'type'])
{'source': 'https://blog.google/outreach-initiatives/sustainability/report-ai-sustainability-google-cop28/', 'title': 'New report from Google: AI and climate action', 'description': 'AI has the potential to mitigate 5-10% of global greenhouse gas emissions according to our new report with Boston Consulting Group.', 'language': 'en-us'}


In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
all_text_splits = text_splitter.split_documents(docs)

In [9]:
from langchain_community.embeddings import GPT4AllEmbeddings
gpt4all_embd = GPT4AllEmbeddings()

bert_load_from_file: gguf version     = 2
bert_load_from_file: gguf alignment   = 32
bert_load_from_file: gguf data offset = 695552
bert_load_from_file: model name           = BERT
bert_load_from_file: model architecture   = bert
bert_load_from_file: model file type      = 1
bert_load_from_file: bert tokenizer vocab = 30522


In [10]:
from langchain.vectorstores import FAISS
vectorstore = FAISS.from_documents(all_text_splits, gpt4all_embd)

In [11]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 1}) # default=4
search= retriever.get_relevant_documents("google AI")
# == vectorstore.similarity_search("....")
print(search)

[Document(page_content='Grow with Google\n                  \n                \n\n\n\n                  Sustainability\n                  \n                \n\n\n\n                   See all\n                \n\n\n\n\n\n            Technology\n            \n\n\n\n\n\n\n\n\n                  AI\n                  \n                \n\n\n\n                  Developers\n                  \n                \n\n\n\n                  Families\n                  \n                \n\n\n\n                  Next billion users\n                  \n                \n\n\n\n                  Safety and security\n                  \n                \n\n\n\n                   See all\n                \n\n\n\n\n\n            Inside Google\n            \n\n\n\n\n\n\n\n\n                  Data centers and infrastructure\n                  \n                \n\n\n\n                  Doodles\n                  \n                \n\n\n\n                  Googlers\n                  \n                \n\n\n

In [12]:
from langchain.schema.runnable import RunnablePassthrough
# from langchain.schema import format_document
from langchain import PromptTemplate
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

llm_prompt = PromptTemplate.from_template(
"""You are an assistant for question-answering tasks.
Use the following context to answer the question.
If you don't know the answer, just say that you don't know.
Use five sentences maximum and keep the answer concise.

Question: {question} 
Context: {context} 
Answer:""")


rag_chain = (
    {
        "context": retriever, 
         "question": RunnablePassthrough()}
    | llm_prompt
    | llm
    | StrOutputParser()
)

In [13]:
%%time
for chunk in rag_chain.stream("How can AI address climate challenges?"):
    print(chunk, end="", flush=True)
print("\n")

1. AI can help optimize energy usage, reduce carbon footprints and improve the efficiency of renewable energy sources by predicting weather patterns more accurately and optimizing energy production schedules.
2. AI algorithms can also assist in developing new green technologies such as sustainable materials and environmentally-friendly products by analyzing data on the impact of different manufacturing processes on the planet's ecosystem.
3. AI can support climate action efforts by providing predictive insights into climate change patterns, allowing us to take preventative measures earlier rather than react later.
4. The use of machine learning algorithms can also be used to predict and model the impacts of extreme weather events such as heat waves, floods or droughts, which are becoming more frequent due to climate change.
5. Finally, AI-powered tools can help businesses reduce their carbon footprint by optimizing supply chains, reducing waste and improving resource efficiency.

CPU t


**Note**: If you want pass an `dict`, make sure the input layer is able to accept it.

In [14]:
%%time
from operator import itemgetter
rag_chain2 = (
    {
        "context": itemgetter("q") |retriever, 
         "question": itemgetter("q")}
    | llm_prompt
    | llm
    | StrOutputParser()
)

for chunk in rag_chain2.stream({"q":"How can AI address climate challenges?"}):
    print(chunk, end="", flush=True)
print("\n")

Artificial Intelligence (AI) can help address climate challenges in several ways, including:
1. Energy efficiency: AI can be used to optimize energy consumption in buildings and factories by analyzing data on energy usage patterns and identifying areas for improvement.
2. Renewable energy production: AI can assist with the planning and operation of renewable energy systems, such as wind and solar farms, by optimizing their performance and reducing downtime.
3. Carbon capture and storage: AI can help monitor and optimize carbon capture and storage (CCS) facilities to increase their efficiency and reduce costs.
4. Climate modeling: AI can be used to improve climate models by analyzing large datasets and identifying patterns that may not be immediately apparent.
5. Sustainable transportation: AI can assist with the development of more sustainable transportation systems, such as electric vehicles and public transit, by optimizing route planning and reducing traffic congestion. 

In summary